In [7]:

import gdown
file_id = '1C5H4xM2nAo0XBe-naUCc-2SKDpjtxXdb'
gdown.download(f'https://drive.google.com/uc?id={file_id}', quiet=False)

FileURLRetrievalError: Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1C5H4xM2nAo0XBe-naUCc-2SKDpjtxXdb

but Gdown can't. Please check connections and permissions.

In [2]:
!pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [3]:
import zipfile

with zipfile.ZipFile('/content/cnsifd_faces_bmp.zip', 'r') as zip_ref:
  zip_ref.extractall('/content/IISCIFD')

In [ ]:
from mxnet import recordio
import mxnet as mx
import matplotlib.pyplot as plt
import numpy as np
import cv2



dataDIR = '/content/Casia/faces_webface_112x112/'


# Define the paths to your train.rec and train.idx files
rec_file = dataDIR+'train.rec'
idx_file = dataDIR+'train.idx'

# Create an MXIndexedRecordIO reader
record = mx.recordio.MXIndexedRecordIO(idx_file, rec_file, 'r')

label_wise_index = dict()

# Read a specific image (e.g., the first image)
max_image_index = 490624


image_index = 1
while(image_index<max_image_index):
  image_record = record.read_idx(image_index)
  header, image = mx.recordio.unpack_img(image_record)
  if(header.label in label_wise_index.keys()):
    label_wise_index[header.label].append(image_index)
  else:
    label_wise_index[header.label] = [image_index]
  image_index+=1

print('success!')

success!


In [ ]:
import pickle
with open('face_data_mapping.pkl','wb') as f:
  pickle.dump(label_wise_index,f)

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!cp  '/content/drive/MyDrive/face_data_mapping.pkl' '/content/'

In [10]:
drive.flush_and_unmount()

In [13]:
import pickle
with open('face_data_mapping.pkl','rb') as f:
  label_wise_index = pickle.load(f)

In [ ]:
imageTrainArr = []
classesTrain = []



for i in label_wise_index.keys():
  classe = str(+i)
  image_index_list = label_wise_index[i]
  for image_index in image_index_list:
      image_record = record.read_idx(image_index)
      header, image = mx.recordio.unpack_img(image_record)
      image = image[:,:,::-1]
      image = cv2.resize(image, dsize=(224, 224), interpolation=cv2.INTER_CUBIC)
      image = np.asarray(image)
      image = image.astype('float32') / 255.0

      imageTrainArr.append(image)
      classesTrain.append(classe)




In [ ]:
cols = ['image', 'person']
trainSet = pd.DataFrame(columns=cols)

In [ ]:
trainSet.image = imageTrainArr
trainSet.person = classesTrain

In [ ]:
import pickle
with open('data_trainCasia.pkl', 'wb') as f:
    pickle.dump(trainSet, f)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


df = pd.read_csv('lfw_allnames.csv')
to_explore = []

for i,j in zip(df.images,df.name):
  if(i>2):
    to_explore.append(j)



In [ ]:
import pandas as pd
import cv2
import numpy as np
import os
from math import floor
from google.colab.patches import cv2_imshow

# Initialing two empty DataFrame with column names
cols = ['image', 'person']
trainSet = pd.DataFrame(columns=cols)

def processImg(path_of_folder):
    face_classifier = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')
    paths = os.listdir(path_of_folder)
    imageTrainArr = []
    classesTrain = []
    for path_of_img in paths:
        varis = path_of_img
        path_of_img = str(path_of_folder + "/" + path_of_img)
        image = cv2.imread(path_of_img)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        try:
          (x, y, w, h) = face_classifier.detectMultiScale(gray, 1.3, 5)[0]
        except:
          continue

        image = image[y:y+h, x:x+w, :]
        img = cv2.resize(image, (224, 224))
        img = np.asarray(img)
        img = img.astype('float32') / 255.0
        l = varis.split('.')
        l[0] = l[0].replace('_', ' ')
        l[0] = l[0][:len(l[0]) - 5]
        imageTrainArr.append(img)
        classesTrain.append(l[0])


    return (imageTrainArr,classesTrain)

inter = '/content/lfw-deepfunneled/lfw-deepfunneled/'

all_images_train = []
all_classes_train = []

for i in to_explore:
  try:
    imageTrainArr,classesTrain = processImg(inter+i)
    all_images_train.extend(imageTrainArr)
    all_classes_train.extend(classesTrain)
  except:
    continue

In [ ]:
print(len(all_images_train))

7288


In [ ]:
print(len(all_classes_train))

7288


In [ ]:
s = set()
for i in all_classes_train:
  s.add(i)
len(s)

901

In [ ]:
trainSet.image = all_images_train
trainSet.person = all_classes_train


In [ ]:
trainSet

,image,person
0,"[[[0.23137255, 0.3529412, 0.48235294], [0.2392...",Aaron Peirsol
1,"[[[0.10980392, 0.30980393, 0.46666667], [0.109...",Aaron Peirsol
2,"[[[0.25490198, 0.4, 0.41568628], [0.23137255, ...",Aaron Peirsol
3,"[[[0.52156866, 0.6431373, 0.654902], [0.482352...",Aaron Peirsol
4,"[[[0.05882353, 0.12941177, 0.21960784], [0.058...",Abdoulaye Wade
...,...,...
7283,"[[[0.16862746, 0.08627451, 0.02745098], [0.164...",Zinedine Zidane
7284,"[[[0.2509804, 0.35686275, 0.43529412], [0.2392...",Zoran Djindjic
7285,"[[[0.2509804, 0.3372549, 0.31764707], [0.25490...",Zoran Djindjic
7286,"[[[0.26666668, 0.3529412, 0.4509804], [0.29019...",Zoran Djindjic


In [ ]:
import pickle
with open('data_train2.pkl', 'wb') as f:
    pickle.dump(trainSet, f)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp data_train.pkl /content/drive/MyDrive/Dataset/data_train2.pkl

In [ ]:
drive.flush_and_unmount()
